# Adjustments rule Jupyter notebook for the STI budgets simulator

In [24]:
# simple trick to make sure that we work in the same directory when we are inside a jupyter notebook and when we are running the simulation from the main directory
import os
import pandas as pd
import datetime
cwd = os.getcwd()
if cwd.endswith('jupyter_notebooks'):
    os.chdir('..')
from rules import adjustments

## Parameters

in order to run the simulations, we need to provide a number of parameters. For the first release, we do not know the parameters in advance. Therefore, the easiest way is to use a dictionary of values.

The list of parameters is the following:

| Name | Description | Optional | Sample value | Default value |
| :---- | :----------- | :------------: | :--- | :--- |
| `start_date` | The start date of the simulation | **No** | 2015-01-01 | |
| `end_date` | The end date of the simulation | **No** | 2097-01-01 | |



In [25]:
parameters = {}
parameters['start_date'] = datetime.datetime(1995,1,1)
parameters['end_date'] = datetime.datetime(2100,1,1)

## boilerplates

In [26]:
df = pd.DataFrame()

## Running the rules

This is were the actual code will calculate the entries of the ledger

In [27]:
df = adjustments.main(params = parameters)

In [28]:
df2 =df.copy(deep=True)
df2.drop(columns=['CF','rule','note'], inplace=True)
df2 = df2.groupby('date').sum()
df2.reset_index(inplace=True)
df2

,date,budget
0,2011-04-30,-34000.000000
1,2011-05-31,-34000.000000
2,2011-06-30,-34000.000000
3,2011-07-31,-34000.000000
4,2011-08-31,-34000.000000
...,...,...
160,2024-08-31,-41666.666667
161,2024-09-30,-41666.666667
162,2024-10-31,-41666.666667
163,2024-11-30,-41666.666667


## Create the vizualization of the returned dataframe

In [29]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
from bokeh.models import ColumnDataSource, CustomJS, NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import RangeSlider
from bokeh.layouts import column

# Base viz configuration
source = ColumnDataSource(df2)
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p=figure(x_axis_type='datetime', tools=TOOLS)
p.title.text = "budget simulation - adjustments - monthly budget"
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "CHF"
p.yaxis.formatter=NumeralTickFormatter(format="0,0")

# Data line
p.line(x='date', y='budget', source=source, line_width=2)

# Hover
hover=HoverTool()
hover.tooltips = [
    ('Date', "@date{%F}"),
    ('Monthly budget', "@budget{'0,0'} CHF"),
    ]
hover.formatters={
    'date': 'datetime',
    'budget': 'numeral',
}
hover.mode='vline'
p.add_tools(hover)

# Rendering of the viz
layout=column(p, sizing_mode='stretch_width')
show(layout)

Loading BokehJS ...